## Structured output

In [14]:
from openai import OpenAI
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv

load_dotenv('../../.env')

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= os.getenv("OPENAI_API_KEY")
)

class RAGGenerationResponse(BaseModel):
    question: str = Field(description="Question asked")
    answer: str = Field(description="The answer to the question")

prompt = """
You are a helpful assistant.
Return an answer to the question.
Question: What is your name?
"""

response = client.responses.parse(
    model="nvidia/nemotron-3-nano-30b-a3b:free",
    input=[
        {"role": "system", "content": prompt}
    ],
    text_format=RAGGenerationResponse,
)

parsed_response = response.output[1].content[0].parsed
print(parsed_response.question)

What is your name?


### RAG Exmple

In [15]:
class RagExample(BaseModel):
    answer:str = Field(description="The answer to te question")

def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_con